In [3]:
import sys
print(sys.executable)

c:\Users\claud\GIT\Proyecto_Hoteles_ETL\venv\Scripts\python.exe


In [4]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
 
# Cargar variables de entorno
load_dotenv()

# URL de la API
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

# Hacer la petición GET a la API
response = requests.get(url)
if response.status_code != 200:
    print("Error al obtener los datos de la API")
    exit()

# Convertir la respuesta a JSON
data = response.json()

# Extraer la lista de eventos
eventos = data.get("@graph", [])

# Fechas de interés
fecha_entrada = datetime.strptime("2025-03-01", "%Y-%m-%d").date()
fecha_salida = datetime.strptime("2025-03-02", "%Y-%m-%d").date()

# Diccionario para almacenar los eventos filtrados
eventos_fechas = {
    "nombre_evento": [],
    "url_evento": [],
    "código_postal": [],
    "dirección": [],
    "horario": [],
    "organización": [],
    "fecha_inicio": [],
    "fecha_fin": []
}

# Función para extraer barrio/distrito desde la URL de `@id`
def extraer_nombre_desde_url(url):
    if url:
        partes = url.split("/")  # Divide la URL por "/"
        return partes[-1] if partes else "No disponible"  # Extrae el último elemento (nombre del distrito/barrio)
    return "No disponible"

# Iterar sobre los eventos
for evento in eventos:
    try:
        fecha_inicio_str = evento.get("dtstart", "")
        fecha_fin_str = evento.get("dtend", fecha_inicio_str)  # Si no hay fecha de fin, asumir que es el mismo día

        # Intentar convertir la fecha (maneja formatos con o sin hora)
        fecha_inicio = datetime.fromisoformat(fecha_inicio_str).date()
        fecha_fin = datetime.fromisoformat(fecha_fin_str).date()

        # Verificar si el evento está dentro del rango de fechas de interés
        if fecha_inicio <= fecha_salida and fecha_fin >= fecha_entrada:
            
            # Manejo de datos de dirección con validaciones
            address = evento.get("address", {})  # Puede no existir
            
            # Extraer datos de código postal y dirección
            codigo_postal = address.get("area", {}).get("postal-code", "No disponible")
            calle = address.get("area", {}).get("street-address", "")

            # Extraer barrio y distrito desde @id
            barrio = extraer_nombre_desde_url(address.get("area", {}).get("@id", ""))
            distrito = extraer_nombre_desde_url(address.get("district", {}).get("@id", ""))

            # Construcción de la dirección
            direccion_partes = [parte for parte in [calle, barrio, distrito] if parte and parte != "No disponible"]
            direccion = ", ".join(direccion_partes) if direccion_partes else "No disponible"

            # **Manejo del horario**
            horario = evento.get("time", "No disponible")  # Si "time" no está, marcarlo como "No disponible"

            # Agregar datos al diccionario
            eventos_fechas["nombre_evento"].append(evento.get("title", "Sin título"))
            eventos_fechas["url_evento"].append(evento.get("link", "Sin URL"))
            eventos_fechas["código_postal"].append(codigo_postal)
            eventos_fechas["dirección"].append(direccion)  # Dirección combinada con calle, barrio y distrito
            eventos_fechas["horario"].append(horario)  # Ahora siempre habrá un horario, o "No disponible"
            eventos_fechas["organización"].append(evento.get("organization", {}).get("organization-name", "No disponible"))
            eventos_fechas["fecha_inicio"].append(fecha_inicio)
            eventos_fechas["fecha_fin"].append(fecha_fin)

    except ValueError:
        continue  # Ignorar eventos con fechas incorrectas

# Convertir a DataFrame
df_eventos = pd.DataFrame(eventos_fechas)
df_eventos

,nombre_evento,url_evento,código_postal,dirección,horario,organización,fecha_inicio,fecha_fin
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,"CALLE SAN JUSTO 5, Palacio, Centro",,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,"PASEO COLOMBIA 1, LosJeronimos, Retiro",,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,"PLAZA LEGAZPI 8, Chopera, Arganzuela",,Matadero Madrid,2024-10-01,2025-06-01
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,No disponible,No disponible,,No disponible,2024-10-14,2025-06-30
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,"AVENIDA MONFORTE DE LEMOS 38, ElPilar, Fuencar...",17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17
...,...,...,...,...,...,...,...,...
290,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,"PASEO CHOPERA 10, Chopera, Arganzuela",18:00,Espacio de Igualdad Juana Doña. Arganzuela,2025-02-18,2025-03-11
291,'El viaje de Leo' de Brunhilde Róman Ibáñez,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,"CALLE SAN JUSTO 5, Palacio, Centro",12:00,Biblioteca Pública Municipal Iván de Vargas (C...,2025-03-02,2025-03-02
292,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,"AVENIDA MONFORTE DE LEMOS 38, ElPilar, Fuencar...",08:30,Biblioteca Pública Municipal José Saramago (Fu...,2025-02-04,2025-03-25
293,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,"CALLE ISAAC ALBENIZ 1, SanIsidro, Carabanchel",19:00,Biblioteca Pública Municipal Ana María Matute ...,2025-02-14,2025-03-13


In [5]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nombre_evento  295 non-null    object
 1   url_evento     295 non-null    object
 2   código_postal  295 non-null    object
 3   dirección      295 non-null    object
 4   horario        295 non-null    object
 5   organización   295 non-null    object
 6   fecha_inicio   295 non-null    object
 7   fecha_fin      295 non-null    object
dtypes: object(8)
memory usage: 18.6+ KB


In [6]:
df_eventos["código_postal"].unique()

array(['28005', '28009', '28045', 'No disponible', '28029', '28022',
       '28014', '28015', '28024', '28011', '28001', '28028', '28042',
       '28036', '28037', '28041', '28025', '28053', '28039', '28002',
       '28004', '28043', '28050', '28007', '28021', '28027', '28008',
       '28012', '28035', '28031', '28049', '28026', '28055', '28054',
       '28019', '28044', '28047', '28017', '28030', '28032', '28020',
       '28038'], dtype=object)

In [7]:
url_fila_3 = df_eventos.loc[3, "url_evento"]
print(url_fila_3)

http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5b58d89ad2232910VgnVCM2000001f4a900aRCRD


Estos eventos se llevan a acbo en varios distritos, por eso tenemos columnas en las que pone "No disponible"

In [8]:
df_eventos.dtypes

nombre_evento    object
url_evento       object
código_postal    object
dirección        object
horario          object
organización     object
fecha_inicio     object
fecha_fin        object
dtype: object

In [9]:
import pandas as pd

def convertir_tipos(df_eventos):
    """
    Convierte las columnas del DataFrame a los tipos adecuados:
    - "código_postal" a INT (sustituyendo "No disponible" por 0).
    - "fecha_inicio" y "fecha_fin" a DATETIME.
    """
    # Convertir "código_postal" a INT (reemplazar "No disponible" con 0)
    df_eventos["código_postal"] = df_eventos["código_postal"].replace("No disponible", 0).astype(int)

    # Convertir "fecha_inicio" y "fecha_fin" a DATETIME
    df_eventos["fecha_inicio"] = pd.to_datetime(df_eventos["fecha_inicio"], errors="coerce")
    df_eventos["fecha_fin"] = pd.to_datetime(df_eventos["fecha_fin"], errors="coerce")

    return df_eventos

# Aplicar la función a df_eventos
df_eventos = convertir_tipos(df_eventos)
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  295 non-null    object        
 1   url_evento     295 non-null    object        
 2   código_postal  295 non-null    int64         
 3   dirección      295 non-null    object        
 4   horario        295 non-null    object        
 5   organización   295 non-null    object        
 6   fecha_inicio   295 non-null    datetime64[ns]
 7   fecha_fin      295 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 18.6+ KB


In [10]:
df_eventos.duplicated().sum()

np.int64(0)

In [11]:
# Renombrar columnas
df_eventos.rename(columns={'código_postal': 'codigo_postal', 'dirección': 'direccion', 'organización': 'organizacion'}, inplace=True)
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  295 non-null    object        
 1   url_evento     295 non-null    object        
 2   codigo_postal  295 non-null    int64         
 3   direccion      295 non-null    object        
 4   horario        295 non-null    object        
 5   organizacion   295 non-null    object        
 6   fecha_inicio   295 non-null    datetime64[ns]
 7   fecha_fin      295 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 18.6+ KB


In [12]:
# Agregar la nueva columna con el valor "Madrid" en todas las filas
df_eventos['nombre_ciudad'] = 'Madrid'
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  295 non-null    object        
 1   url_evento     295 non-null    object        
 2   codigo_postal  295 non-null    int64         
 3   direccion      295 non-null    object        
 4   horario        295 non-null    object        
 5   organizacion   295 non-null    object        
 6   fecha_inicio   295 non-null    datetime64[ns]
 7   fecha_fin      295 non-null    datetime64[ns]
 8   nombre_ciudad  295 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 20.9+ KB


In [13]:
df_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,nombre_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,"CALLE SAN JUSTO 5, Palacio, Centro",,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,"PASEO COLOMBIA 1, LosJeronimos, Retiro",,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,"PLAZA LEGAZPI 8, Chopera, Arganzuela",,Matadero Madrid,2024-10-01,2025-06-01,Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,No disponible,,No disponible,2024-10-14,2025-06-30,Madrid
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,"AVENIDA MONFORTE DE LEMOS 38, ElPilar, Fuencar...",17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid


In [14]:
df_eventos.to_pickle("../data/data_raw/eventos_madrid.pkl")